In [1]:
from pyspark import SparkContext, SparkConf #spark
import pydoop.hdfs as hdfs #for saving local file as hdfs 
from pyspark.sql import SQLContext # for data frame
import urllib #for imitating hdfs auto escape string
import re #for escape string 
from operator import methodcaller

In [2]:
import pandas as pd

In [3]:
conf = SparkConf().setAppName("error_machine_part_2")
sc = SparkContext(conf=conf)

In [4]:
def Count_Lines(df_path):
    rdd_hdfs = sc.textFile(df_path)
    count_lines = rdd_hdfs.count()
    return count_lines

In [5]:
def Word_Count_Each_Text(df_path):
    rdd_hdfs = sc.textFile(df_path)
    count_lines = rdd_hdfs.count()
    # spliting by special characters and number
    rdd_split_map = rdd_hdfs.flatMap(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)
    #ready to count
    rdd_count_map = rdd_split_map.map(lambda x: (x, 1))
    #mapping each key and counting it
    res_reduce = rdd_count_map.reduceByKey(lambda a, b: a+b )
    # desc sorting by counting number
    res_reduce = res_reduce.sortBy(lambda c: c[1], False)
    return res_reduce

In [6]:
def Pick_Top_Text(rdd_wc_reduce, count_lines):
    #picking string which always show and counting bigger than file lines  (deviation value 2)
    pick_top_string_map = rdd_wc_reduce.filter(lambda x: x[1] >= count_lines-2)
    pick_top_string_map.collect()
    return pick_top_string_map.collect()

In [7]:
def Pick_Last_Text(rdd_wc_reduce, count_lines):
    pick_last_string_map = rdd_wc_reduce.filter(lambda x: x[1] < count_lines-2)
    pick_last_string_map = pick_last_string_map.sortBy(lambda c: c[1], False)
    return pick_last_string_map.collect()

In [8]:
def Filter_Top_Texts(line, list_tops):
    res_str = ""
    #filter all the top text 
    for text in line:
        flag = 0
        if str(text) == "":
            continue;
        for top in list_tops:
            #res_str +='----' +top[0]
            if top[0] == str(text):

                flag = 1
                break;
        if(flag == 1):
            continue;
        else:
            res_str +="," + str(text)
    return res_str[1:]

In [9]:
def Delete_Top_Text_for_Each_Line(df_path, top_collect):
    rdd_hdfs = sc.textFile(df_path)
    # spliting by special characters
    rdd_sp_map = rdd_hdfs.map(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)
    #filter all the top text 
    rdd_fil_map = rdd_sp_map.map(lambda x:Filter_Top_Texts(x, top_collect) )
    return rdd_fil_map

In [258]:
def Group_Map_by_top_key(line, last_tops_collect):
    res_str = ""
    find_key = ""
    #filter all the top text 
    for top in last_tops_collect:
        res_str = ""
        for text in line:
            if (str(top[0].upper()) == str(text.upper()) and find_key == ""):
                find_key = str(text)
            else:
                res_str +="," + str(text)
        if(find_key != ""):
            break;
    #(mainKey,"val0,val1,....")
    return (find_key,res_str) # 加排名

In [259]:
#for compare
def Group_Map_by_normal_key(line, normal_last_tops_collect, self_last_tops_collect):
    res_str = ""
    find_key = ""
    tag = "noFind"
    #filter all the top text 
    for top in normal_last_tops_collect:
        res_str = ""
        for text in line:
            if (str(top[0]) == str(text) and find_key == ""):
                find_key = str(text)
                tag = "Find"
            else:
                res_str +="," + str(text)
        if(find_key != ""):
            break;
    if(find_key == ""):
        find_key,res_str = Group_Map_by_top_key(line, self_last_tops_collect)
    return (tag, find_key,res_str)

In [260]:
#for compare
def MapReduce_Each_Line_by_normal_Top_key(comapre_last_tops_collect, normal_last_tops_collect, delete_top_map):
    #map by key
    rdd_key_map = delete_top_map.map(lambda x:Group_Map_by_normal_key(x.split(','),normal_last_tops_collect, comapre_last_tops_collect))
    rdd_find_map = rdd_key_map.map(lambda x: ( x[1],x[2]) if x[0] == "Find" else ('',''))
    rdd_no_find_map = rdd_key_map.map(lambda x: ( x[1],x[2]) if x[0] == "noFind" else ('',''))
    #rdd_find_red = rdd_find_map.reduceByKey(lambda a,b:a+b)
    #rdd_no_find_red = rdd_no_find_map.reduceByKey(lambda a,b:a+b)
    return (rdd_find_map, rdd_no_find_map)

In [261]:
def MapReduce_Each_Line_by_Other_Top_key(last_tops_collect, delete_top_map):
    #map by key
    rdd_key_map = delete_top_map.map(lambda x:Group_Map_by_top_key(x.split(','),last_tops_collect))
    rdd_key_red = rdd_key_map.sortBy(lambda c: c[0], False).reduceByKey(lambda a,b: a+b)
    return rdd_key_red

In [509]:
def Values_Distinct(rdd_red_by_top_key, list_or_dist):
    #distinct value
    dist_temp = {}
    line_temp = []
    rdd_collect = rdd_red_by_top_key.collect()
    for k,v in rdd_collect:
        rdd_temp = sc.parallelize(v.split(','))
        rdd_dis_val_map = rdd_temp.map(lambda x: (x,1))
        rdd_dis_val_red = rdd_dis_val_map.reduceByKey(lambda a,b: a+b)
        rdd_dis_fn = rdd_dis_val_red.map(lambda x: x[0])
        #print rdd_dis_fn.collect()
        if(list_or_dist == 'dist'):
            dist_temp[k] = rdd_dis_fn.collect()
        else:
            line_temp.append((k,rdd_dis_fn.collect()))
    if(list_or_dist == 'dist'):
        return dist_temp
    else:
        return line_temp


In [510]:
def put_values_in_top_dist(rdd_key_value, col_tops):
    dist_top_all_values = {}
    for i in col_normal_last_top:
        dist_top_all_dup_values[i[0]] = ""
        print i[0]

In [511]:
def Convert_List_to_Dist(list):
    dist_temp = {}
    dist_temp[v[0].upper()] = rdd_dis_fn.collect()

In [512]:
#for compare
def Normal_Last_Tops(input_path):
    count_lines = Count_Lines(input_path)
    red_wordcount = Word_Count_Each_Text(input_path)
    col_red_last =  Pick_Last_Text(red_wordcount, count_lines)
    return col_red_last

In [513]:
def Main_Normal_Case(input_path):
    count_lines = Count_Lines(input_path)
    red_wordcount = Word_Count_Each_Text(input_path)
    col_red_top = Pick_Top_Text(red_wordcount, count_lines)
    col_red_last =  Pick_Last_Text(red_wordcount, count_lines)
    rdd_map_filiter_top = Delete_Top_Text_for_Each_Line(inputPath_normal, col_red_top)
    rdd_red_by_last_top_key = MapReduce_Each_Line_by_Other_Top_key(col_red_last, rdd_map_filiter_top)
    list_key_val_distinct = Values_Distinct(rdd_red_by_last_top_key, 'dist')
    return list_key_val_distinct
   

# normal log

In [514]:
inputPath_normal = '/user/spark/input/log_normal'
dist_normal = Main_Normal_Case(inputPath_normal)

In [515]:
dist_normal

{'CHANGE': ['',
  'a',
  'c',
  'e',
  'vethb',
  'NETDEV',
  'abef',
  'IPv',
  'ready',
  'link',
  'docker',
  'efdc',
  'kernel',
  'vethc',
  'vethfba',
  'ef',
  'ADDRCONF',
  'becomes',
  'f',
  'vethe',
  'veth',
  'vethcc',
  'b',
  'vethfd',
  'eb'],
 'DBus': ['', 'from', 'dnsmasq', 'upstream', 'setting', 'servers'],
 'Dispatcher': ['',
  'systemd',
  'Service',
  'Started',
  'Manager',
  'Starting',
  'Network',
  'Script'],
 'INFO': ['',
  'detected',
  'sched',
  'self',
  'kernel',
  'tasks',
  'on',
  'rcu',
  'CPUs',
  'stall',
  'stalls',
  'CPU'],
 'NetworkManager': ['',
  'and',
  'domain',
  'managed',
  'ifupdown',
  'control',
  'ca',
  'activated',
  'none',
  'udev',
  'detection',
  'device',
  'manager',
  'done',
  'link',
  'yes',
  'connected',
  'activating',
  'efdc',
  'successful',
  'networking',
  'for',
  'becf',
  'support',
  'sys',
  'add',
  'aa',
  'fb',
  'started',
  'state',
  'Wired',
  'audit',
  'dns',
  'policy',
  'CONNECTING',
  'net',

In [516]:
a = dist_normal.collect()
for k,v in a:
    if ("Network" in v):
        print k 
        
        
        
         (4, 'info', 'NetworkManager'),
 (5, 'info', 'NetworkManager,plen'),
 (6, 'info', 'NetworkManager,gateway'),
 (7, 'info', 'NetworkManager,server,identifier'),
 (8,
  'systemd',
  'dbus,system,Activating,via,name,org,freedesktop,nm,dispatcher,unit,dbus,org,freedesktop,nm,dispatcher'),
 (9, 'info', 'NetworkManager,lease'),
 (10, 'info', 'NetworkManager'),
 (11, 'info', 'NetworkManager,domain,name,localdomain'),
 (12, 'info', 'NetworkManager,wins'),
 (13, 'info', 'NetworkManager,dhcp,ens,state,changed,bound,bound'),
 (14, 'systemd', 'Network,Script,Dispatcher'),
 (15,
  'org',
  'dbus,system,Successfully,activated,service,freedesktop,nm,dispatcher'),
 (16, 'systemd', 'Network,Script,Dispatcher'),
 (17, 'ens', 'nm,dispatcher,req,dhcp,change,request,scripts'),
 (18, 'ens', 'nm,dispatcher,req,dhcp,change,start,running,ordered,scripts'),
 (20, 'd', 'dhclient,DHCPREQUEST,of,on,ens,to,xid,x'),
 (21, 'info', 'NetworkManager'),
 (22, 'info', 'NetworkManager,plen'),
 (23, 'info', 'NetworkManager,gateway'),
 (24, 'info', 'NetworkManager,server,identifier'),
 (25, 'info', 'NetworkManager,lease'),
 (26, 'info', 'NetworkManager'),
 (27, 'info', 'NetworkManager,domain,name,localdomain'),
 (28, 'info', 'NetworkManager,wins'),
 (29, 'info', 'NetworkManager,dhcp,ens,state,changed,bound,bound'),
 (30,
  'systemd',
  'dbus,system,Activating,via,name,org,freedesktop,nm,dispatc

AttributeError: 'dict' object has no attribute 'collect'

In [371]:
#for keyword in dist_normal:
#    rdd = sc.parallelize(dist_normal[keyword])
#    rdd.saveAsTextFile("/user/spark/output/temp1/"+keyword)

In [543]:
for keyword in dist_normal:
    if ("NetworkManager" in dist_normal[keyword]):
        print keyword

NetworkManager


In [402]:
count_lines = Count_Lines(inputPath_normal)

In [403]:
red_wordcount = Word_Count_Each_Text(inputPath_normal)

In [404]:
col_red_top = Pick_Top_Text(red_wordcount, count_lines)

In [405]:
col_red_top[:10]

[(u'', 483648),
 (u'virtual', 11546),
 (u'jerry', 11365),
 (u'Jan', 11364),
 (u'machine', 11364)]

In [406]:
col_red_last =  Pick_Last_Text(red_wordcount, count_lines)

In [407]:
col_red_last[20:]

[(u'change', 866),
 (u'Service', 858),
 (u'port', 857),
 (u'req', 856),
 (u'scripts', 856),
 (u'Manager', 855),
 (u'name', 855),
 (u'veth', 855),
 (u'system', 855),
 (u'Network', 854),
 (u'Dispatcher', 854),
 (u'Script', 854),
 (u'from', 771),
 (u'seconds', 746),
 (u'xid', 726),
 (u'renewal', 724),
 (u'DHCPREQUEST', 724),
 (u'DHCPACK', 724),
 (u'time', 691),
 (u'kernel', 641),
 (u'state', 583),
 (u'b', 581),
 (u'f', 563),
 (u'address', 555),
 (u'e', 543),
 (u'new', 511),
 (u'nameserver', 508),
 (u'device', 489),
 (u'c', 481),
 (u'CUPS', 469),
 (u'Starting', 459),
 (u'Scheduler', 457),
 (u'Successfully', 448),
 (u'start', 435),
 (u'activated', 433),
 (u'running', 433),
 (u'server', 433),
 (u'request', 432),
 (u'ordered', 428),
 (u'changed', 428),
 (u'Activating', 427),
 (u'avahi', 427),
 (u'unit', 427),
 (u'via', 427),
 (u'domain', 425),
 (u'localdomain', 425),
 (u'gateway', 425),
 (u'wins', 425),
 (u'identifier', 425),
 (u'plen', 425),
 (u'lease', 425),
 (u'etc', 409),
 (u'root', 401),

In [408]:
rdd_map_filiter_top = Delete_Top_Text_for_Each_Line(inputPath_normal, col_red_top)

In [409]:
rdd_map_filiter_top.take(30)

['anacron,Job,cron,daily,terminated',
 'anacron,Normal,exit,job,run',
 'dhclient,DHCPREQUEST,of,on,ens,to,port,xid,x,da,d,b',
 'dhclient,DHCPACK,of,from',
 'NetworkManager,info,address',
 'NetworkManager,info,plen',
 'NetworkManager,info,gateway',
 'NetworkManager,info,server,identifier',
 'dbus,system,Activating,via,systemd,service,name,org,freedesktop,nm,dispatcher,unit,dbus,org,freedesktop,nm,dispatcher,service',
 'NetworkManager,info,lease,time',
 'dhclient,bound,to,renewal,in,seconds',
 'NetworkManager,info,nameserver',
 'NetworkManager,info,domain,name,localdomain',
 'NetworkManager,info,wins',
 'NetworkManager,info,dhcp,ens,state,changed,bound,bound',
 'systemd,Starting,Network,Manager,Script,Dispatcher,Service',
 'dbus,system,Successfully,activated,service,org,freedesktop,nm,dispatcher',
 'systemd,Started,Network,Manager,Script,Dispatcher,Service',
 'nm,dispatcher,req,dhcp,change,ens,new,request,scripts',
 'nm,dispatcher,req,dhcp,change,ens,start,running,ordered,scripts',
 'sys

In [410]:
rdd_red_by_last_top_key = MapReduce_Each_Line_by_Other_Top_key(col_red_last, rdd_map_filiter_top)

In [411]:
rdd_red_by_last_top_key.take(10)

[('cron',
  ',anacron,Job,daily,terminated,anacron,Job,daily,terminated,anacron,Job,weekly,terminated,anacron,Job,daily,terminated,anacron,Job,daily,terminated,mailing,output'),
 ('device',
  ',libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,message,repeated,times,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,message,repeated,times,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,ethtool,ioctl,error,No,such,libvirtd,message,repeated,times,ethtool,ioctl,error,No,su

In [416]:
a = rdd_red_by_last_top_key.collect()
for k,v in a:
    if ("Network" in v):
        print k 

NetworkManager
Dispatcher


In [500]:
#distinct value
dist_temp = {}
line_temp = []
rdd_collect = rdd_red_by_last_top_key.collect()
for k,v in rdd_collect:
    if(k == 'Dispatcher'):
        rdd_temp = sc.parallelize(v.split(','))
        print rdd_temp.take(4)
        rdd_dis_val_map = rdd_temp.map(lambda x: (x,1))
        print rdd_dis_val_map.take(4)
        rdd_dis_val_red = rdd_dis_val_map.reduceByKey(lambda a,b: a+b)
        print rdd_dis_val_red.take(4)
        rdd_dis_fn = rdd_dis_val_red.map(lambda x: x[0])
        print rdd_dis_fn.collect()
        #print rdd_dis_fn.collect()

        break;


['', 'systemd', 'Starting', 'Network']
[('', 1), ('systemd', 1), ('Starting', 1), ('Network', 1)]
[('', 1), ('systemd', 854), ('Service', 854), ('Started', 427)]
['', 'systemd', 'Service', 'Started', 'Manager', 'Starting', 'Network', 'Script']


In [ ]:
print dist_normal['Dispatcher'.upper()]

In [413]:
df_col = rdd_red_by_last_top_key.collect()

In [385]:
dfff = rdd_red_by_last_top_key.toDF()

In [386]:
dfff.show()

+----------+--------------------+
|        _1|                  _2|
+----------+--------------------+
|      cron|,anacron,Job,dail...|
|    device|,libvirtd,ethtool...|
|   manager|,gnome,session,ER...|
|      smbd|               ,done|
|    CHANGE|,kernel,IPv,ADDRC...|
|   systemd|,Started,CUPS,Sch...|
|      from|,kernel,eth,renam...|
|       lib|,gnome,session,va...|
|      DBus|,dnsmasq,setting,...|
|      exit|,anacron,Normal,j...|
|      INFO|,kernel,rcu,sched...|
|   started|,anacron,Job,cron...|
|  whoopsie|            ,offline|
|    daemon|,usr,lib,snapd,sn...|
|       not|,gnome,session,ad...|
|       The|,rtkit,daemon,can...|
|nameserver|,dnsmasq,using,dn...|
|      info|,dockerd,time,T,l...|
|         c|,avahi,daemon,Int...|
|         e|,gnome,session,ad...|
+----------+--------------------+
only showing top 20 rows



In [283]:
for i in df_col: 
    if(i[0] == "NetworkManager"):
        print i


('NetworkManager', ',info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins,info,dhcp,ens,state,changed,bound,bound,info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins,info,dhcp,ens,state,changed,bound,bound,info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins,info,dhcp,ens,state,changed,bound,bound,info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins,info,dhcp,ens,state,changed,bound,bound,info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins,info,dhcp,ens,state,changed,bound,bound,info,address,info,plen,info,gateway,info,server,identifier,info,lease,time,info,nameserver,info,domain,name,localdomain,info,wins

In [284]:
dist_top_all_values = {}
for i in col_normal_last_top:
    dist_top_all_values[i[0]] = ""
    print i[0]
    break;
#print dist_top_all_values

col_normal_last_top

   

NetworkManager


[(u'NetworkManager', 4390),
 (u'info', 4327),
 (u'dhclient', 2483),
 (u'dispatcher', 2137),
 (u'nm', 2137),
 (u'ens', 2044),
 (u'systemd', 1992),
 (u'to', 1579),
 (u'bound', 1574),
 (u'dhcp', 1573),
 (u'of', 1480),
 (u'org', 1359),
 (u'freedesktop', 1341),
 (u'service', 1290),
 (u'dbus', 1281),
 (u'x', 952),
 (u'on', 945),
 (u'Started', 933),
 (u'd', 886),
 (u'in', 868),
 (u'change', 866),
 (u'Service', 858),
 (u'port', 857),
 (u'req', 856),
 (u'scripts', 856),
 (u'Manager', 855),
 (u'name', 855),
 (u'veth', 855),
 (u'system', 855),
 (u'Network', 854),
 (u'Dispatcher', 854),
 (u'Script', 854),
 (u'from', 771),
 (u'seconds', 746),
 (u'xid', 726),
 (u'renewal', 724),
 (u'DHCPREQUEST', 724),
 (u'DHCPACK', 724),
 (u'time', 691),
 (u'kernel', 641),
 (u'state', 583),
 (u'b', 581),
 (u'f', 563),
 (u'address', 555),
 (u'e', 543),
 (u'new', 511),
 (u'nameserver', 508),
 (u'device', 489),
 (u'c', 481),
 (u'CUPS', 469),
 (u'Starting', 459),
 (u'Scheduler', 457),
 (u'Successfully', 448),
 (u'start

In [285]:
df =  pd.DataFrame(df_col )

In [286]:
df

,0,1
0,cron,",anacron,Job,daily,terminated,anacron,Job,dail..."
1,device,",libvirtd,ethtool,ioctl,error,No,such,libvirtd..."
2,manager,",gnome,session,ERROR,flashback,monitor,c,make,..."
3,smbd,",done"
4,CHANGE,",kernel,IPv,ADDRCONF,NETDEV,vethe,abef,link,be..."
5,systemd,",Started,CUPS,Scheduler,Started,CUPS,Scheduler..."
6,from,",kernel,eth,renamed,vethbbd,a,kernel,vethbbd,a..."
7,lib,",gnome,session,var,dpkg,lock"
8,DBus,",dnsmasq,setting,upstream,servers,from,dnsmasq..."
9,exit,",anacron,Normal,job,run,anacron,Normal,jobs,ru..."


In [287]:
for i in df[0]:
    if 'NetworkManager' in i:
        print "yes"
    else:
        print 'no'

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no


In [288]:
red = rdd_key_map.reduceByKey(lambda a,b: a+b)

In [289]:
red.take(3)

[('', '')]

In [290]:
rdd_red_by_last_top_key

PythonRDD[3109] at collect at <ipython-input-279-bbd793124725>:1

In [291]:
df_col = rdd_red_by_last_top_key.collect()

In [292]:
df =  pd.DataFrame(df_col )

In [293]:
df

,0,1
0,cron,",anacron,Job,daily,terminated,anacron,Job,dail..."
1,device,",libvirtd,ethtool,ioctl,error,No,such,libvirtd..."
2,manager,",gnome,session,ERROR,flashback,monitor,c,make,..."
3,smbd,",done"
4,CHANGE,",kernel,IPv,ADDRCONF,NETDEV,vethe,abef,link,be..."
5,systemd,",Started,CUPS,Scheduler,Started,CUPS,Scheduler..."
6,from,",kernel,eth,renamed,vethbbd,a,kernel,vethbbd,a..."
7,lib,",gnome,session,var,dpkg,lock"
8,DBus,",dnsmasq,setting,upstream,servers,from,dnsmasq..."
9,exit,",anacron,Normal,job,run,anacron,Normal,jobs,ru..."


In [415]:
for i in df[0]:
    if 'NetworkManager' in i:
        print  df[1]
    else:
        print 'no'
    break

no


# compare error log

In [295]:
def Main_Compare_Case(input_path, normal_path):
    count_lines = Count_Lines(input_path)
    red_wordcount = Word_Count_Each_Text(input_path)
    col_red_top = Pick_Top_Text(red_wordcount, count_lines)
    #col_red_last =  Pick_Last_Text(red_wordcount, count_lines)
    rdd_map_filiter_top = Delete_Top_Text_for_Each_Line(inputPath_normal, col_red_top)
    
    col_normal_last_top = Normal_Last_Tops(normal_path)
    rdd_red_by_last_top_key = MapReduce_Each_Line_by_Other_Top_key(col_normal_last_top, rdd_map_filiter_top)
    list_key_val_distinct = Values_Distinct(rdd_red_by_last_top_key, 'line')
    return list_key_val_distinct

In [296]:
inputPath_error = '/user/spark/input/log_error'
res_compare = Main_Compare_Case(inputPath_error,inputPath_normal)

In [297]:
print res_compare

[('CRON', ['', 'Job', 'mailing', 'daily', 'terminated', 'weekly', 'anacron', 'output']), ('DEVICE', ['', 'ethtool', 'ioctl', 'repeated', 'libvirtd', 'message', 'such', 'times', 'error', 'No']), ('MANAGER', ['', 'c', 'crtc', 'make', 'logical', 'flashback', 'config', 'monitor', 'assertion', 'failed', 'session', 'ERROR', 'output', 'NULL', 'gnome']), ('SMBD', ['', 'done']), ('CHANGE', ['', 'a', 'c', 'e', 'vethb', 'NETDEV', 'abef', 'IPv', 'ready', 'link', 'docker', 'efdc', 'kernel', 'vethc', 'vethfba', 'ef', 'ADDRCONF', 'becomes', 'f', 'vethe', 'veth', 'vethcc', 'b', 'vethfd', 'eb']), ('SYSTEMD', ['', 'aa', 'OpenBSD', 'file', 'Service', 'process', 'Make', 'Reloaded', 'snaps', 'apt', 'abef', 'Stopping', 'Duplicate', 'timer', 'Closed', 'lib', 'scheduling', 'jobs', 'efdc', 'Docker', 'Engine', 'systemd', 'LSB', 'dump', 'for', 'Started', 'becf', 's', 'been', 'Login', 'ms', 'state', 'tmpfiles', 'Automatically', 'Scheduler', 'var', 'FAN', 'entered', 'Could', 'Starting', 'daily', 'available', 'a', 

In [298]:
count_lines = Count_Lines(inputPath_error)
print count_lines

1943


In [299]:
red_wordcount = Word_Count_Each_Text(inputPath_error)
col_red_top = Pick_Top_Text(red_wordcount, count_lines)
print col_red_top[:5]

[(u'', 79117), (u'machine', 1943), (u'Jan', 1943), (u'virtual', 1943), (u'jerry', 1943)]


In [300]:
col_compare_last_top =  Pick_Last_Text(red_wordcount, count_lines)

In [301]:
col_compare_last_top[:5]

[(u'NetworkManager', 534),
 (u'info', 529),
 (u'kernel', 425),
 (u'dhclient', 383),
 (u'ens', 337)]

In [302]:
rdd_map_filiter_top = Delete_Top_Text_for_Each_Line(inputPath_normal, col_red_top)
rdd_map_filiter_top.take(2)

['anacron,Job,cron,daily,terminated', 'anacron,Normal,exit,job,run']

In [303]:
col_normal_last_top = Normal_Last_Tops(inputPath_normal)
print col_normal_last_top[:3]

[(u'NetworkManager', 4390), (u'info', 4327), (u'dhclient', 2483)]


In [304]:
rdd_key_map = rdd_map_filiter_top.map(lambda x:Group_Map_by_normal_key(x.split(','),col_normal_last_top, col_compare_last_top))

In [305]:
rdd_key_map.count()

11364

In [306]:
rdd_key_map.take(5)

[('Find', 'cron', ',anacron,Job,daily,terminated'),
 ('Find', 'exit', ',anacron,Normal,job,run'),
 ('Find', 'dhclient', ',DHCPREQUEST,of,on,ens,to,port,xid,x,da,d,b'),
 ('Find', 'dhclient', ',DHCPACK,of,from'),
 ('Find', 'NetworkManager', ',info,address')]

In [307]:
rdd_key_map = rdd_key_map.map(lambda x: ( x[1],x[2]) if x[0] == "Find" else ('',''))

In [308]:
rdd_key_map = rdd_key_map.map(lambda x: ( x[1],x[2]) if x[0] == "noFind" else ('',''))

In [309]:
rdd_key_map.count()

11364

In [310]:
(rdd_red_find_by_last_top_key, rdd_red_no_find_by_last_top_key) = MapReduce_Each_Line_by_normal_Top_key(col_compare_last_top, col_normal_last_top, rdd_map_filiter_top)
rdd_red_find_by_last_top_key.count()


11364

In [311]:
rdd_red_find_by_last_top_key.take(3)

[('cron', ',anacron,Job,daily,terminated'),
 ('exit', ',anacron,Normal,job,run'),
 ('dhclient', ',DHCPREQUEST,of,on,ens,to,port,xid,x,da,d,b')]

In [312]:
rdd_red_no_find_by_last_top_key.count()

11364

In [313]:
dist_find_distinct_com = Values_Distinct(rdd_red_find_by_last_top_key, 'dist')

KeyboardInterrupt: 

In [ ]:
list_find_distinct_com = Values_Distinct(rdd_red_find_by_last_top_key, 'list')

In [314]:
rdd_red_find_by_last_top_key.take(2)

[('cron', ',anacron,Job,daily,terminated'),
 ('exit', ',anacron,Normal,job,run')]

In [315]:
print dist_find_distinct_com["INFO"]

['ac', 'ae', 'ff', 'process', 'enabled', 'is', 'ea', 'Daemon', 'ec', 'assigned', 'conf', 'updockerd', 'aeff', 'terminateddockerd', 'bfe', 'nameservers', 'addressdockerd', 'content', 'cc', 'fada', 'msg', 'containers', 'dde', 'non', 'run', 'Default', 'completed', 'db', 'daemon', 'preferred', 'var', 'running', 'IPv', 'Layer', 'DNS', 'Docker', 'with', 'docker', 'bip', 'a', 'c', 'e', 'default', 'signal', 'bef', 'fb', 'startdockerd', 'versiondockerd', 'localhost', 'stopping', 'fd', 'nameserver', 'addressability', 'Adding', 'Loading', 'receiving', 'ad', 'ead', 'No', 'Graph', 'fe', 'ef', 'cf', 'sockdockerd', 'servers', 'set', 'migration', 'dockerd', 'initializationdockerd', 'aufs', 'after', 'option', 'are', 'terminated', 'bdf', 'listen', 'bfd', 'containerd', 'left', 'Processing', 'to', 'libcontainerd', 'eaeba', 'Using', 'piddockerd', 'ddd', 'bridge', 'be', 'used', 'an', 'ba', 'dcfddf', 'IP', 'de', 'Firewalld', 'fc', 'took', 'fa', 'b', 'cleaned', 'T', 'new', 'falsedockerd', 'ab', 'nameserverdoc

In [316]:
dist_no_find_distinct_com = Values_Distinct(rdd_red_no_find_by_last_top_key, 'dist')
print list_no_find_distinct_com

KeyboardInterrupt: 

In [317]:
rdd_key_map_ = rdd_map_filiter_top.map(lambda x:Group_Map_by_top_key(x.split(','),list_find_distinct_com))

In [318]:
rdd_key_map_.take(2)

[('cron', ',anacron,Job,daily,terminated'),
 ('exit', ',anacron,Normal,job,run')]

In [319]:
rdd_key_red_ = rdd_key_map_.reduceByKey(lambda a,b:a+b)

In [320]:
rdd_key_red_.take(10)

[('INFO',
  ',kernel,rcu,sched,detected,stalls,on,CPUs,tasks,kernel,rcu,sched,self,detected,stall,on,CPU'),
 ('cron',
  ',anacron,Job,daily,terminated,anacron,Will,run,job,daily,in,min,anacron,Will,run,job,weekly,in,min,anacron,Job,daily,started,anacron,Updated,timestamp,for,job,daily,to,anacron,Job,daily,terminated,anacron,Job,weekly,started,anacron,Updated,timestamp,for,job,weekly,to,anacron,Job,weekly,terminated,anacron,Will,run,job,daily,in,min,anacron,Job,daily,started,anacron,Updated,timestamp,for,job,daily,to,anacron,Job,daily,terminated,anacron,Will,run,job,daily,in,min,anacron,Job,daily,started,anacron,Updated,timestamp,for,job,daily,to,anacron,Job,daily,terminated,mailing,output,anacron,Will,run,job,daily,in,min,anacron,Job,daily,started,anacron,Updated,timestamp,for,job,daily,to'),
 ('systemd',
  ',dbus,system,Activating,via,service,name,org,freedesktop,nm,dispatcher,unit,dbus,org,freedesktop,nm,dispatcher,service,Starting,Network,Manager,Script,Dispatcher,Service,Started,Ne

In [321]:
print dist_normal['NetworkManager']

KeyError: 'NetworkManager'

In [518]:
pdf = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dist_normal.iteritems() ]))

In [519]:
#df = df.reindex(index=df.index[::-1])

In [520]:
pdf

,CHANGE,DBus,Dispatcher,INFO,NetworkManager,New,The,WARNING,anacron,avahi,...,org,port,smbd,started,systemd,time,to,veth,whoopsie,x
0,,,,,,,,,,,...,,,,,,,,,,
1,a,from,systemd,detected,and,daemon,daemon,process,sequentially,daemon,...,GLib,a,done,cron,aa,real,GLib,aa,offline,interrupt
2,c,dnsmasq,Service,sched,domain,for,thread,was,Jobs,for,...,srollout,c,NaN,Job,OpenBSD,daemon,aa,is,NaN,readahead
3,e,upstream,Started,self,managed,c,is,dead,will,mDNS,...,WARN,e,NaN,daily,file,slave,xd,device,NaN,c
4,vethb,setting,Manager,kernel,ifupdown,mDNS,starving,already,be,abef,...,G,vethb,NaN,anacron,Service,unauthorized,cron,ready,NaN,RIP
5,NETDEV,servers,Starting,tasks,control,abef,canary,binary,executed,IPv,...,for,abef,NaN,weekly,process,for,vethbbd,efdc,NaN,init
6,abef,NaN,Network,on,ca,IPv,action,session,NaN,Interface,...,terminal,efdc,NaN,NaN,Make,not,daily,from,NaN,local
7,IPv,NaN,Script,rcu,activated,interface,apparently,gnome,NaN,docker,...,did,state,NaN,NaN,Reloaded,image,file,for,NaN,handle
8,ready,NaN,NaN,CPUs,none,avahi,Taking,Child,NaN,vethe,...,IS,disabled,NaN,NaN,snaps,continuing,ffffffff,Interface,NaN,invoke
9,link,NaN,NaN,stall,udev,docker,rtkit,NaN,NaN,relevant,...,Terminal,docker,NaN,NaN,apt,msg,open,entered,NaN,df


In [521]:
for i in  pdf['SYSTEMD']:
    if('Network' == i):
        print 'ok'
        

KeyError: 'SYSTEMD'

In [522]:
res_pdf = pdf.query('SYSTEMD == "Network"')

UndefinedVariableError: name 'SYSTEMD' is not defined

In [523]:
res_pdf

,ANACRON,AVAHI,B,BY,C,CHANGE,COM,CRON,D,DAEMON,...,SMBD,STARTED,SYSTEMD,THE,TIME,TO,VETH,WARNING,WHOOPSIE,X
23,NaN,NaN,NaN,NaN,NaN,vethfd,NaN,NaN,afed,NaN,...,NaN,NaN,dump,NaN,Handler,match,link,NaN,NaN,cron


In [524]:
len(res_pdf)

1

In [525]:
r_col = rdd_red_find_by_last_top_key.collect()

In [526]:
r_col

[('cron', ',anacron,Job,daily,terminated'),
 ('exit', ',anacron,Normal,job,run'),
 ('dhclient', ',DHCPREQUEST,of,on,ens,to,port,xid,x,da,d,b'),
 ('dhclient', ',DHCPACK,of,from'),
 ('NetworkManager', ',info,address'),
 ('NetworkManager', ',info,plen'),
 ('NetworkManager', ',info,gateway'),
 ('NetworkManager', ',info,server,identifier'),
 ('dispatcher',
  ',dbus,system,Activating,via,systemd,service,name,org,freedesktop,nm,unit,dbus,org,freedesktop,nm,dispatcher,service'),
 ('NetworkManager', ',info,lease,time'),
 ('dhclient', ',bound,to,renewal,in,seconds'),
 ('NetworkManager', ',info,nameserver'),
 ('NetworkManager', ',info,domain,name,localdomain'),
 ('NetworkManager', ',info,wins'),
 ('NetworkManager', ',info,dhcp,ens,state,changed,bound,bound'),
 ('systemd', ',Starting,Network,Manager,Script,Dispatcher,Service'),
 ('dispatcher',
  ',dbus,system,Successfully,activated,service,org,freedesktop,nm'),
 ('systemd', ',Started,Network,Manager,Script,Dispatcher,Service'),
 ('dispatcher', ',n

In [527]:
no_group = []
i = 0
for line in r_col:
    no_text = ""
    for text in line[1].split(','):
        res_pdf = pdf.query(' '+line[0].upper()+' == @text')
        if (len(res_pdf) < 1):
            no_text +="," +text
    i += 1
    #print line[0]
    #print no_text
    if (no_text != ""):
        print line[1]
        print ' '+line[0].upper()+' == @text'
        no_group.append((i, line[0].upper(), no_text[1:]))
    if (i == 20):
        break;

UndefinedVariableError: name 'CRON' is not defined

In [528]:
no_group


[]

In [336]:
pdf['SYSTEMD']

0                
1              aa
2         OpenBSD
3            file
4         Service
5         process
6            Make
7        Reloaded
8           snaps
9             apt
10           abef
11       Stopping
12      Duplicate
13          timer
14         Closed
15            lib
16     scheduling
17           jobs
18           efdc
19         Docker
20         Engine
21        systemd
22            LSB
23           dump
24            for
25        Started
26           becf
27              s
28           been
29          Login
          ...    
145        Secure
146        Failed
147             h
148          CIFS
149       changed
150            up
151        server
152        vethfd
153       network
154           API
155          ABRT
156           min
157            of
158       locally
159        dumped
160            or
161       address
162         udevd
163           NaN
164           NaN
165           NaN
166           NaN
167           NaN
168           NaN
169       

In [529]:
col_compares = rdd_key_map_.collect()


In [541]:

no_group = []
i = 0
for line in col_compares:
    try:
        no_text = ""
        for text in line[1].split(','):
            res_pdf = pdf.query(' '+line[0] +' == @text')
            if (len(res_pdf) < 1):
                no_text +="," +text
        i += 1
        print line[0]
        print no_text
        if (no_text != ""):
            no_group.append((i, line[0] , no_text[1:]))
        if (i == 600):
            break;
    except:
        print "error" +line[0]

cron

exit

d
,dhclient,DHCPREQUEST,of,on,ens,to,xid,x
errorfrom
info
,NetworkManager
info
,NetworkManager,plen
info
,NetworkManager,gateway
info
,NetworkManager,server,identifier
systemd
,dbus,system,Activating,via,name,org,freedesktop,nm,dispatcher,unit,dbus,org,freedesktop,nm,dispatcher
info
,NetworkManager,lease
errorin
info
,NetworkManager
info
,NetworkManager,domain,name,localdomain
info
,NetworkManager,wins
info
,NetworkManager,dhcp,ens,state,changed,bound,bound
systemd
,Network,Script,Dispatcher
org
,dbus,system,Successfully,activated,service,freedesktop,nm,dispatcher
systemd
,Network,Script,Dispatcher
ens
,nm,dispatcher,req,dhcp,change,request,scripts
ens
,nm,dispatcher,req,dhcp,change,start,running,ordered,scripts
systemd

d
,dhclient,DHCPREQUEST,of,on,ens,to,xid,x
errorfrom
errorin
info
,NetworkManager
info
,NetworkManager,plen
info
,NetworkManager,gateway
info
,NetworkManager,server,identifier
info
,NetworkManager,lease
info
,NetworkManager
info
,NetworkManager,domain,name,

In [542]:
no_group

[(3, 'd', 'dhclient,DHCPREQUEST,of,on,ens,to,xid,x'),
 (4, 'info', 'NetworkManager'),
 (5, 'info', 'NetworkManager,plen'),
 (6, 'info', 'NetworkManager,gateway'),
 (7, 'info', 'NetworkManager,server,identifier'),
 (8,
  'systemd',
  'dbus,system,Activating,via,name,org,freedesktop,nm,dispatcher,unit,dbus,org,freedesktop,nm,dispatcher'),
 (9, 'info', 'NetworkManager,lease'),
 (10, 'info', 'NetworkManager'),
 (11, 'info', 'NetworkManager,domain,name,localdomain'),
 (12, 'info', 'NetworkManager,wins'),
 (13, 'info', 'NetworkManager,dhcp,ens,state,changed,bound,bound'),
 (14, 'systemd', 'Network,Script,Dispatcher'),
 (15,
  'org',
  'dbus,system,Successfully,activated,service,freedesktop,nm,dispatcher'),
 (16, 'systemd', 'Network,Script,Dispatcher'),
 (17, 'ens', 'nm,dispatcher,req,dhcp,change,request,scripts'),
 (18, 'ens', 'nm,dispatcher,req,dhcp,change,start,running,ordered,scripts'),
 (20, 'd', 'dhclient,DHCPREQUEST,of,on,ens,to,xid,x'),
 (21, 'info', 'NetworkManager'),
 (22, 'info', '

In [ ]:
res_pdf = pdf.query('INFO == "NetworkManager"')

In [ ]:
res_pdf

In [192]:
d = {'rt':'2323r','sdf':"sdfsdf",'2222':"eee"}

In [193]:
d

{'2222': 'eee', 'rt': '2323r', 'sdf': 'sdfsdf'}

In [194]:
for i in d:
    print d[i]

2323r
sdfsdf
eee


In [196]:
c ={}

In [197]:
 for i in d:
    c[i] = d[i]

In [198]:
c

{'2222': 'eee', 'rt': '2323r', 'sdf': 'sdfsdf'}

In [201]:
col_normal_last_top

[(u'NetworkManager', 4390),
 (u'info', 4327),
 (u'dhclient', 2483),
 (u'dispatcher', 2137),
 (u'nm', 2137),
 (u'ens', 2044),
 (u'systemd', 1992),
 (u'to', 1579),
 (u'bound', 1574),
 (u'dhcp', 1573),
 (u'of', 1480),
 (u'org', 1359),
 (u'freedesktop', 1341),
 (u'service', 1290),
 (u'dbus', 1281),
 (u'x', 952),
 (u'on', 945),
 (u'Started', 933),
 (u'd', 886),
 (u'in', 868),
 (u'change', 866),
 (u'Service', 858),
 (u'port', 857),
 (u'req', 856),
 (u'scripts', 856),
 (u'Manager', 855),
 (u'name', 855),
 (u'veth', 855),
 (u'system', 855),
 (u'Network', 854),
 (u'Dispatcher', 854),
 (u'Script', 854),
 (u'from', 771),
 (u'seconds', 746),
 (u'xid', 726),
 (u'renewal', 724),
 (u'DHCPREQUEST', 724),
 (u'DHCPACK', 724),
 (u'time', 691),
 (u'kernel', 641),
 (u'state', 583),
 (u'b', 581),
 (u'f', 563),
 (u'address', 555),
 (u'e', 543),
 (u'new', 511),
 (u'nameserver', 508),
 (u'device', 489),
 (u'c', 481),
 (u'CUPS', 469),
 (u'Starting', 459),
 (u'Scheduler', 457),
 (u'Successfully', 448),
 (u'start

In [203]:
dist_top_all_dup_values = {}
for i in col_normal_last_top:
    dist_top_all_dup_values[i[0]] = ""
    print i[0]
    break;
    

NetworkManager
